# Interpolación

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import RegularGridInterpolator

In [9]:
df = pd.read_csv("dataframe2.csv")

In [10]:
# Paso 1: Definir las coordenadas únicas ordenadas
x_sorted = np.sort(df['x'].unique())
y_sorted = np.sort(df['y'].unique())
z_sorted = np.sort(df['z'].unique())

In [11]:
# Verifica que haya suficientes puntos únicos en cada dimensión para formar una cuadrícula
if len(x_sorted) < 2 or len(y_sorted) < 2 or len(z_sorted) < 2:
    raise ValueError("No hay suficientes valores únicos en las coordenadas x, y o z para una cuadrícula.")

In [ ]:
# Paso 2: Generar puntos uniformemente distribuidos para interpolar
# Número de puntos a generar
n_points = 50000
points_new = np.column_stack((
    np.random.uniform(x_sorted.min(), x_sorted.max(), n_points),
    np.random.uniform(y_sorted.min(), y_sorted.max(), n_points),
    np.random.uniform(z_sorted.min(), z_sorted.max(), n_points)
))

In [12]:
# Paso 3: Interpolación trilineal para cada columna de energía
energy_columns = df.columns[3:]  # Ajusta esto si tus columnas de energía tienen otro rango
interpolated_energies = []

for column in energy_columns:
    # Crear una cuadrícula 3D completa para las energías
    grid_values = np.full(
        (len(x_sorted), len(y_sorted), len(z_sorted)), 
        np.nan
    )
    
    # Rellenar la cuadrícula con los valores correspondientes
    for _, row in df.iterrows():
        x_idx = np.where(x_sorted == row['x'])[0][0]
        y_idx = np.where(y_sorted == row['y'])[0][0]
        z_idx = np.where(z_sorted == row['z'])[0][0]
        grid_values[x_idx, y_idx, z_idx] = row[column]
    
    # Verifica si hay valores NaN en la cuadrícula
    if np.isnan(grid_values).any():
        raise ValueError(f"La cuadrícula 3D para la columna '{column}' contiene valores NaN.")

    # Crear el interpolador trilineal
    interpolator = RegularGridInterpolator(
        (x_sorted, y_sorted, z_sorted),  # Coordenadas ordenadas
        grid_values,  # Valores de energía
        bounds_error=False, fill_value=None  # Permitir extrapolación fuera de los límites
    )
    
    # Interpolar para los nuevos puntos
    interpolated_energy = interpolator(points_new)
    interpolated_energies.append(interpolated_energy)

In [13]:
# Paso 4: Crear un nuevo DataFrame con los resultados interpolados
result_df = pd.DataFrame(
    points_new, columns=['x', 'y', 'z']  # Coordenadas interpoladas
)

# Agregar las energías interpoladas como columnas
for i, column in enumerate(energy_columns):
    result_df[f'interpolated_{column}'] = interpolated_energies[i]

/tmp/ipykernel_255516/3794745808.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[f'interpolated_{column}'] = interpolated_energies[i]
/tmp/ipykernel_255516/3794745808.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[f'interpolated_{column}'] = interpolated_energies[i]
/tmp/ipykernel_255516/3794745808.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat

In [14]:
# Mostrar el resultado
print(result_df.head())

          x         y         z  interpolated_valor_1  interpolated_valor_2  \
0 -0.452160 -0.407454 -0.423024                   0.0                   0.0   
1  0.378952  0.978523  0.279634                   0.0                   0.0   
2 -0.974306  0.479246 -0.860636                   0.0                   0.0   
3 -0.145220 -0.888070 -0.687144                   0.0                   0.0   
4 -0.866021 -0.475819  0.282003                   0.0                   0.0   

   interpolated_valor_3  interpolated_valor_4  interpolated_valor_5  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   interpolated_valor_6  interpolated_valor_7  ...  interpolated_valor_191  \
0                   

In [15]:
result_df

,x,y,z,interpolated_valor_1,interpolated_valor_2,interpolated_valor_3,interpolated_valor_4,interpolated_valor_5,interpolated_valor_6,interpolated_valor_7,...,interpolated_valor_191,interpolated_valor_192,interpolated_valor_193,interpolated_valor_194,interpolated_valor_195,interpolated_valor_196,interpolated_valor_197,interpolated_valor_198,interpolated_valor_199,interpolated_valor_200
0,-0.452160,-0.407454,-0.423024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000357,0.000346,0.000335,0.000325,0.000315,0.000305,0.000296,0.000286,0.000278,0.000269
1,0.378952,0.978523,0.279634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000382,0.000370,0.000358,0.000347,0.000337,0.000326,0.000316,0.000306,0.000297,0.000287
2,-0.974306,0.479246,-0.860636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000356,0.000345,0.000334,0.000324,0.000314,0.000304,0.000295,0.000286,0.000277,0.000268
3,-0.145220,-0.888070,-0.687144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000353,0.000342,0.000331,0.000321,0.000311,0.000301,0.000292,0.000283,0.000274,0.000266
4,-0.866021,-0.475819,0.282003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000360,0.000348,0.000338,0.000327,0.000317,0.000307,0.000298,0.000288,0.000279,0.000271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-0.935348,0.245806,-0.479346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000359,0.000347,0.000337,0.000326,0.000316,0.000306,0.000297,0.000288,0.000279,0.000270
49996,0.027475,-0.175395,0.944229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000375,0.000364,0.000352,0.000341,0.000331,0.000320,0.000311,0.000301,0.000292,0.000283
49997,-0.524597,0.610342,-0.766277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000362,0.000351,0.000340,0.000330,0.000320,0.000310,0.000300,0.000291,0.000282,0.000273
49998,-0.767076,-0.049696,0.858900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000369,0.000358,0.000347,0.000336,0.000325,0.000315,0.000306,0.000296,0.000287,0.000278


In [16]:
# Si deseas guardar los resultados en un archivo CSV
result_df.to_csv('interpolated_results.csv', index=False)